In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
synthetic_df = pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/Heart Disease/synthetic cleveland.csv')

In [4]:
synthetic_df['target'] = np.where(synthetic_df['target']==0, 0, 1)
synthetic_df = synthetic_df.fillna(0)
x = synthetic_df.drop('target',axis = 1 ).iloc[:].to_numpy()

y = synthetic_df['target'].to_numpy()

In [5]:
from typing import List, Tuple, Dict
from copy import copy

from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.base import clone, BaseEstimator

try:
    import tensorflow as tf
except (ModuleNotFoundError, ImportError):
    import warnings

    warnings.warn("Tensorflow is not installed")

class ShadowModels:
    """
    Creates a swarm of shadow models and trains them with a split
    of the synthetic data.
    Parameters
    ----------
    X: ndarray or DataFrame
    y: ndarray or str
        if X it's a DataFrame then y must be the target column name,
        otherwise 
    n_models: int
        number of shadow models to build. Higher number returns
        better results but is limited by the number of records 
        in the input data.
    target_classes: int
        number of classes of the target model or lenght of the
        prediction array of the target model.
    learner: learner? #fix type
        learner to use as shadow model. It must be as similar as 
        possible to the target model. It must have `predict_proba` 
        method. Now only sklearn learners are implemented.
    Returns
    -------
    ShadowModels object
    """

    def __init__(
        self,
        X: np.ndarray,
        y: np.ndarray,
        n_models: int,
        target_classes: int,
        learner,
        **fit_kwargs,
    ) -> None:

        self.n_models = n_models
        self.X = X
        if self.X.ndim > 1:
            # flatten images or matrices inside 1rst axis
            self.X = self.X.reshape(self.X.shape[0], -1)

        self.y = y
        self.target_classes = target_classes
        self._splits = self._split_data(self.X, self.y, self.n_models, self.target_classes)
        self.learner = learner
        self.models = self._make_model_list(self.learner, self.n_models)

        # train models
        self.results = self.train_predict_shadows(**fit_kwargs)

    @staticmethod
    def _split_data(
        X: np.ndarray, y: np.ndarray, n_splits: int, n_classes: int
    ) -> List[np.ndarray]:
        """
        Split manually into n datasets maintaining class proportions
        """
        # data = np.hstack((data[0], data[1].reshape(-1, 1)))
        # X = data
        # y = data[:, -1]
        classes = range(n_classes)
        class_partitions = []
        # Split by class
        for clss in classes:

            X_clss = X[y == clss]
            y_clss = y[y == clss]
            batch_size = len(X_clss) // n_splits
            splits = []
            for i in range(n_splits):
                split_X = X_clss[i * batch_size : (i + 1) * batch_size, :]
                split_y = y_clss[i * batch_size : (i + 1) * batch_size]
                splits.append(np.hstack((split_X, split_y.reshape(-1, 1))))
            class_partitions.append(splits)

        # -------------------
        # consolidate splits into ndarrays
        # -------------------

        grouped = []
        for split in range(n_splits):
            parts = []
            for part in class_partitions:
                parts.append(part[split])
            grouped.append(parts)

        splits = []
        for group in grouped:
            splits.append(np.vstack(group))

        return splits

    @staticmethod
    def _make_model_list(learner, n) -> List:
        """
        Intances n shadow models, copies of the input parameter learner
        """
        try:
            if isinstance(learner, tf.keras.models.Model):
                models = [copy(learner) for _ in range(n)]
        except NameError:
            print("using sklearn shadow models")
            pass

        if isinstance(learner, BaseEstimator):
            models = [clone(learner) for _ in range(n)]

        return models

    def train_predict_shadows(self, **fit_kwargs):
        """
        "in" : 1
        "out" : 0
        """

        # TRAIN and predict
        results = []
        for model, data_subset in tqdm_notebook(zip(self.models, self._splits)):
            X = data_subset[:, :-1]
            y = data_subset[:, -1]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

            model.fit(X_train, y_train, **fit_kwargs)
            # data IN training set labelet 1
            y_train = y_train.reshape(-1, 1)
            predict_in = model.predict_proba(X_train)
            res_in = np.hstack((predict_in, y_train, np.ones_like(y_train)))

            # data OUT of training set, labeled 0
            y_test = y_test.reshape(-1, 1)
            predict_out = model.predict_proba(X_test)
            print(predict_out)
            res_out = np.hstack((predict_out, y_test, np.zeros_like(y_test)))

            # concat in single array
            model_results = np.vstack((res_in, res_out))
            results.append(model_results)

        results = np.vstack(results)
        return results

    def __repr__(self):
        rep = (
            f"Shadow models: {self.n_models}, {self.learner.__class__}\n"
            f"lengths of data splits : {[len(s) for s in self._splits]}"
        )
        return rep

In [6]:
logisticRegr = LogisticRegression(solver='liblinear')
sh = ShadowModels(x, y, 5, 3, logisticRegr)
shadow_data = sh.results

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:134: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

[[9.18835397e-01 8.11646028e-02]
 [9.38821811e-01 6.11781892e-02]
 [3.47689951e-03 9.96523100e-01]
 [3.64205956e-03 9.96357940e-01]
 [9.19929918e-01 8.00700825e-02]
 [9.12752478e-01 8.72475217e-02]
 [9.03938621e-01 9.60613792e-02]
 [1.05116212e-01 8.94883788e-01]
 [2.69122392e-01 7.30877608e-01]
 [9.85315629e-01 1.46843713e-02]
 [1.29849022e-01 8.70150978e-01]
 [2.68128979e-01 7.31871021e-01]
 [9.89694125e-01 1.03058750e-02]
 [2.48344129e-02 9.75165587e-01]
 [9.58863024e-01 4.11369757e-02]
 [3.61910156e-04 9.99638090e-01]
 [9.94976795e-01 5.02320481e-03]
 [9.72194656e-01 2.78053445e-02]
 [2.63405758e-02 9.73659424e-01]
 [9.25491544e-01 7.45084558e-02]
 [9.86698008e-01 1.33019917e-02]
 [1.13007659e-01 8.86992341e-01]
 [9.97026833e-01 2.97316725e-03]
 [8.76629436e-01 1.23370564e-01]
 [9.69752338e-02 9.03024766e-01]
 [9.83587368e-01 1.64126315e-02]
 [8.12586981e-01 1.87413019e-01]
 [6.25905631e-03 9.93740944e-01]
 [9.95310039e-01 4.68996111e-03]
 [9.65081976e-01 3.49180237e-02]]
[[9.47492

In [7]:
shadow_data

array([[0.60906347, 0.39093653, 0.        , 1.        ],
       [0.97204294, 0.02795706, 0.        , 1.        ],
       [0.06653541, 0.93346459, 1.        , 1.        ],
       ...,
       [0.78869991, 0.21130009, 0.        , 0.        ],
       [0.72708819, 0.27291181, 1.        , 0.        ],
       [0.02329123, 0.97670877, 1.        , 0.        ]])

In [8]:
class AttackModels:
    def __init__(self, target_classes, attack_learner):
        """
        Attacker models to learn class membership from shadow data.
        Parameters
        ----------
        target_classes: int
            number of classes that the target model can predict
        attack_learning: learner
            trainable learner to model memebership from shadow data.
            The learner its cloned into n models, one for each target class,
            and each model is trained on a class subset of the shadow data.
        Returns
        -------
        AttackModels class instance
        """
        self.target_classes = target_classes
        self.attack_learner = attack_learner
        # 1 model for each class
        self.attack_models = [clone(self.attack_learner) for _ in range(target_classes)]

        self._fited = False

    @staticmethod
    def _update_learner_params(learner, **learner_params) -> None:
        # safety check if dict is well formed
        for k in learner_params.keys():
            if not hasattr(learner, k):
                raise AttributeError(
                    f"Learner parameter {k} is not an attribute of {learner.__class__}"
                )

        # update learner params
        learner.__dict__.update(**learner_params)

    def fit(self, shadow_data, **learner_kwargs) -> None:
        """
        Trains `attack_models` with `shadow_data`. Each model is trained with
        with a subset of the same class of `shadow_data`.
        Parameters
        ----------
        shadow_data: np.ndarray
            Shadow data. Results from `ShadowModels`.
            Last column (`[:,-1]`) must be the membership label of the shadow
            prediction, where 1 means that the record was present in the 
            shadow training set ('in') and 0 if the recored was in the test
            set ('out').
            Second last column (`[:,-2]`) must be the data class. this will
            be used as grouper to split the data for each attack model.
            The rest of the columns are the class probability vector
            predicted by the shadow model.
        Returns
        -------
        None
        TODO
        ----
            Tweak model params with something like **learner_kwargs
            cross-validate
            grid search?
        """
        # split data into subsets, n == target_classes
        membership_label = shadow_data[:, -1]
        class_label = shadow_data[:, -2]
        data = shadow_data[:, :-2]
        for i, model in enumerate(self.attack_models):
            X = data[class_label == i]
            y = membership_label[class_label == i]

            # update model params
            self._update_learner_params(model, **learner_kwargs)
            # train model
            model.fit(X, y)

        self._fited = True

    def predict(self, X, y, batch=False) -> np.ndarray:
        """
        Predicts if `X` is real member of `y` in the attacked
        private training set.
        Parameters
        ----------
        X: np.ndarray
            Probability vector result from target model
        y: int, np.ndarray
            estimated class of the data record used to get `X`
        """
        if not self._fited:
            print("Must run `fit` method first")
            return

        if not batch:
            model_cls = y
            model = self.attack_models[model_cls]
            prob_vec = model.predict_proba(X)

            if y == np.argmax(prob_vec) and np.argmax(prob_vec) == 1:
                return 1

            else:
                return 0

        elif batch:

            model_classes = np.unique(y)
            res = []
            for model_cls in model_classes:
                X_cls = X[y == model_cls]
                model = self.attack_models[model_cls]
                attack_res = model.predict_proba(X_cls)
                res.append(attack_res)

            return np.concatenate(res)

In [9]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
attacker = AttackModels(target_classes=2, attack_learner=model)
attacker.fit(shadow_data)

# Membership attack on decision tree

In [10]:
df =  pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/Heart Disease/cleveland.csv', header = None)
df[13] = np.where(df[13]==0, 0, 1)
df = df.fillna(0)
x = df.drop(13,axis = 1 )
y = df[13]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
clf = DecisionTreeClassifier(random_state=0,max_leaf_nodes = 50).fit(x_train,y_train)

In [11]:
y_pred_train = clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 100.0
accuracy on test data is 72.0


In [12]:
X_in = clf.predict_proba(x_train)
res_in = attacker.predict(X_in, y_pred_train, batch = True)

In [13]:
X_out = clf.predict_proba(x_test)
res_out = attacker.predict(X_out, y_test, batch=True)

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = np.concatenate((np.argmax(res_in, axis=1), np.argmax(res_out, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [15]:
precision_score(y_true, y_pred)

0.6923076923076923

In [16]:
recall_score(y_true, y_pred)

0.4433497536945813

In [17]:
f1_score(y_true, y_pred)

0.5405405405405406

# Membership attack on the target using Random Foest Classifier


In [18]:
from sklearn.ensemble import RandomForestClassifier
r_clf = RandomForestClassifier(n_estimators = 4,random_state=0)
r_clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=4, random_state=0)

In [19]:
y_pred_train = r_clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = r_clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 94.08866995073892
accuracy on test data is 73.0


In [20]:
X_in = r_clf.predict_proba(x_train)
res_input = attacker.predict(X_in, y_train, batch=True)

In [21]:
X_out = r_clf.predict_proba(x_test)
res_output = attacker.predict(X_out, y_test, batch=True)

In [22]:
y_pred = np.concatenate((np.argmax(res_input, axis=1), np.argmax(res_output, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [23]:
precision_score(y_true, y_pred)

0.6809815950920245

In [24]:
recall_score(y_true, y_pred)

0.5467980295566502

In [25]:
f1_score(y_true, y_pred)

0.6065573770491802

# Membership inference attack on stacking classifier

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svr', make_pipeline(StandardScaler(),
                          LinearSVC(random_state=42)))
]
s_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)


s_clf.fit(x_train, y_train)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('svr',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('linearsvc',
                                                 LinearSVC(random_state=42))]))],
                   final_estimator=LogisticRegression())

In [27]:
y_pred_train = s_clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = s_clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 91.13300492610837
accuracy on test data is 87.0


In [28]:
X_in = s_clf.predict_proba(x_train)
r_input = attacker.predict(X_in, y_train, batch=True)

In [29]:
X_out = s_clf.predict_proba(x_test)
r_output = attacker.predict(X_out, y_test, batch=True)

In [30]:
y_pred = np.concatenate((np.argmax(r_input, axis=1), np.argmax(r_output, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [31]:
precision_score(y_true, y_pred)

0.6818181818181818

In [32]:
recall_score(y_true, y_pred)

0.5911330049261084

In [33]:
f1_score(y_true, y_pred)

0.633245382585752